# Lecture 1-2: Logistic Regression (for classification)

.### impliment here: ###

## 0. Kaggle Glace https://www.kaggle.com
## 1. Apply Your Kaggle Account 
## 2. Get the API key(kaggle.json) and upload


.### impliment here: ###

### Place your Kaggle API key properly

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

### download dataset by API for https://www.kaggle.com/tongpython/cat-and-dog

In [0]:
### impliment your code here: ###

!kaggle datasets download -d 

### impliment your code here: ###

### decompress the dataset -> jpgs

In [0]:
!unzip cat-and-dog.zip

### Libraies

In [0]:
import os
from collections import Counter

import sklearn
import numpy as np
import pandas as pd

import cv2
import matplotlib.pyplot as plt

### parse images into DataFrame

In [0]:
def dataset_parse(folders, size=(32, 32)):
    
    if isinstance(folders, str):
        folders = [folders, ]

    features = list()
    labels = list()

    for folder in folders:
        for root, dirs, files in os.walk(folder):
            for f in filter(lambda x: x.endswith('.jpg'), files):
                img = cv2.resize(cv2.imread(os.path.join(root, f)), size)
                label = os.path.basename(root)
                
                labels.append(label)
                features.append(img)

    df = pd.DataFrame({'img':features, 'category': labels})
    df = df.sample(n=len(df), random_state=0) # shuffle
    df.reset_index(drop=True, inplace=True)
    return df

In [0]:
data_test ='test_set/test_set/'
data_train ='training_set/training_set/'

# In this Task, we merge original train & test set and split by requirement(you can deside depends on the task)

In [0]:
df = dataset_parse([data_train, data_test], size=(32,32))

# Glace of the data

In [0]:
df.head(5)

In [0]:
Counter(df['category'])

In [0]:
index = 0
df['img'][index]

In [0]:
index = 0
print('This is an image of "{}" as size: {}'.format(df['category'][index], df['img'][index].shape))
plt.imshow(df['img'][index])
plt.show()

In [0]:
print('max value if the dataset: {}'.format(np.max(np.concatenate(df['img'].values))))
print('min value if the dataset: {}'.format(np.min(np.concatenate(df['img'].values))))

### We don't know how to process array yet. So we convert array into vactor with definded order.

In [0]:
df['feature'] = df['img'].apply(lambda x: x.reshape([1, -1]))

### Normalize the feature (you can try what is the difference)

In [0]:
### impliment your code here: ###

df['feature'] = df['feature'].apply(lambda x: )

### impliment your code here: ###

### We don't know how to predict "text", so convert cats->1, dogs->0

In [0]:
### impliment your code here: ###

df['label'] = df['category'].apply(lambda x: )

### impliment your code here: ###

### train/test split with 80/20%

In [0]:
train_rate = 0.8
df_train = df[:int(train_rate * len(df))]
df_test = df[int(train_rate * len(df)):]

### split feature and label

In [0]:
x_train = np.concatenate(df_train['feature'].values, axis=0)
y_train = df_train['label'].values

x_test = np.concatenate(df_test['feature'].values, axis=0)
y_test = df_test['label'].values

# Logistic Regression

![alt text](https://drive.google.com/uc?id=1-LYvA_aT5UpjTAzzIW128XHMcDz0RKCZ)
reference: Jessee Kung 2019/4/10 (https://sites.google.com/view/tu-ad1062-mlfundamentals/)

# f: activation function
# we set f(z) = sigmoid(z) for Linear Regression (https://en.wikipedia.org/wiki/Sigmoid_function)

![alt text](https://drive.google.com/uc?id=1CO5oExjpjal7Mqf2oGrozedmwXBw6Pkk)

![alt text](https://drive.google.com/uc?id=1pAav0St8ajL7vx0iO1cUb6amV_aD_2Qc)

In [0]:
class GDLogisticRegression:

    def __init__(self, lr=0.03, it=1000):
        self.w = None
        self.b = None
        self.lr = lr
        self.iteration = it

    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    def fit(self, x, y, verbose=100):
        y = np.array(y)
        losses = list()
        
        print(x.shape)
        m, n = x.shape
        self.w = np.zeros([n])
        self.b = 0

        for i in range(self.iteration):
            _y = self.predict(x)
            
            loss = (- 1 / m) * np.sum(y*np.log(_y) + (1-y)*np.log(1-_y))
            losses.append(loss)

            if verbose and not (i%verbose):
                print('iteration: {}, training loss: {:.6f}'.format(i, loss))

            dw = (1 / m) * np.dot(x.T, (_y - y))
            db = np.mean(_y - y)

            self.w -= self.lr * dw
            self.b -= self.lr * db
        # return losses

    def predict(self, x, ):
        assert self.w is not None
        assert self.b is not None
        x = np.array(x)
        if len(x.shape)==1:
            x = np.concatenate(x, axis=0)
        return self.sigmoid(np.dot(self.w, x.T) + self.b)


## trainning

In [0]:
clf = GDLogisticRegression(lr=0.01)

In [0]:
clf.fit(x_train, y_train)

In [0]:
def binarize_predict(_y_pred, thresh=0.5):
    return (_y_pred > thresh).astype(int)

In [0]:
from sklearn.metrics import accuracy_score

y_pred_train = binarize_predict(clf.predict(x_train))
accuracy_train = accuracy_score(y_pred_train, y_train)

y_pred_test = binarize_predict(clf.predict(x_test))
accuracy_test = accuracy_score(y_pred_test, y_test)

print(clf)
print('training accuracy: {:4f}, testing accuracy: {:4f}'.format(accuracy_train, accuracy_test))

### training a sklearn model

In [0]:
from sklearn.linear_model import SGDClassifier

In [0]:
clf = SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                    early_stopping=False, epsilon=0.1, eta0=0.1, fit_intercept=True,
                    l1_ratio=0.15, learning_rate='constant', loss='log',
                    max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
                    power_t=0.5, random_state=None, shuffle=True, tol=0.001,
                    validation_fraction=0.1, verbose=0, warm_start=False)
clf.fit(x_train, y_train)
accuracy_train = accuracy_score(binarize_predict(clf.predict(x_train)), y_train)
accuracy_test = accuracy_score(binarize_predict(clf.predict(x_test)), y_test)

print('training accuracy: {:4f}, testing accuracy: {:4f}'.format(accuracy_train, accuracy_test))

# developement(validation) set

### split the developement set from training set

In [0]:
dev_rate = 0.1

### impliment your code here: ###

_x_dev = [:int(dev_rate * len(x_train))]
_y_dev = [:int(dev_rate * len(x_train))]

_x_train = [int(dev_rate * len(x_train)):]
_y_train = [int(dev_rate * len(x_train)):]

### impliment your code here: ###

train on _train and select the best parameter/model by _dev

In [0]:
best_score = 0
best_clf = None 

### impliment your code here: ###

for clf in []:

    clf.fit
    score = 

    print('val score: {}, \nmodel: {}\n'.format(score, clf))

    if score > best_score:
        best_clf = clf
        best_score = score

### impliment your code here: ###
print('\n\n\n')
print('WE TAKE THE BEST MODEL as \n val score: {}, \nmodel: {}'.format(best_score, best_clf))
        

In [0]:
accuracy_test = accuracy_score(binarize_predict(best_clf.predict(x_test)), y_test)
print('testing score: {}'.format(accuracy_test))

# You should also know:

*   recall
*   precision
*   f1 score
*   false positive rate(FPR)
*   ROC / AUC
*   cross validation
*   overfitting/ underfitting
*   other sklearn classification models




